In [1]:
cd "c:\Users\ecometto001\Documents\Personal\Tesis"

c:\Users\ecometto001\Documents\Personal\Tesis


In [61]:
import pandas as pd
import numpy as np
from typing import List, Optional

import statsmodels.api as sm

# plt.style.use('dark_background')
random_state=42

In [3]:
tesis = pd.read_csv("Datos/tesis_final.csv")
tesis.head()

,idPaciente,tpo_programa,TAS,Adherencia,Peso,Altura,IMC,DBT,Sexo,Edad,Fuma,ant_HTA,tas_basal,ICC,Adherencia_Acumulada,Adherencia_Total,TAS_Media_Acumulada
0,4026,0,119.000000,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,119.000000
1,4026,1,127.000000,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,123.000000
2,4026,2,140.000000,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,128.666667
3,4026,3,146.712710,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,133.178178
4,4026,4,177.708084,1,82.0,152.0,35.0,0.0,0,76.0,0.0,1,116,0.852459,1.0,1.0,142.084159


In [4]:
tesis.isnull().sum()

idPaciente                0
tpo_programa              0
TAS                       0
Adherencia                0
Peso                     63
Altura                   63
IMC                      63
DBT                     126
Sexo                      0
Edad                    119
Fuma                    126
ant_HTA                   0
tas_basal                 0
ICC                     266
Adherencia_Acumulada      0
Adherencia_Total          0
TAS_Media_Acumulada       0
dtype: int64

### Imputar valores faltantes

In [5]:
categorical = ["Adherencia", "DBT", "Sexo", "Fuma"]
numerical = ["tpo_programa", "TAS", "Peso", "Altura", "IMC", "Edad", "ICC", "tas_basal", "Adherencia_Acumulada", "Adherencia_Total"]

In [6]:
from statistics import mean, median, mode

for cat in categorical:
    tesis[cat].fillna(mode(tesis[tesis["tpo_programa"] == 0][cat]), inplace=True)

for num in numerical:
    tesis[num].fillna(np.mean(tesis[tesis["tpo_programa"] == 0][num]), inplace=True)

In [7]:
tesis["Adherencia_lag1"] = tesis.groupby("idPaciente")["Adherencia"].shift(1).fillna(0)
tesis["Adherencia_lag2"] = tesis.groupby("idPaciente")["Adherencia"].shift(2).fillna(0)
tesis["TAS_lag1"] = tesis.groupby("idPaciente")["TAS"].shift(1).fillna(tesis["tas_basal"])
tesis["TAS_lag2"] = tesis.groupby("idPaciente")["TAS"].shift(2).fillna(tesis["tas_basal"])

In [8]:
from sklearn.preprocessing import PolynomialFeatures

columns = tesis.columns

poly = PolynomialFeatures(interaction_only=True, degree=3)
tesis = poly.fit_transform(tesis)
new_columns = poly.get_feature_names(columns)
new_columns = [column.replace(' ', '*') for column in new_columns]

tesis = pd.DataFrame(tesis, columns=new_columns)

tesis['Intercept'] = 1

c:\Users\ecometto001\Documents\Personal\Tesis\.venv\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### Probar efectos aleatorios

In [9]:
fixed_effects = [
    'Intercept', 'Sexo', 'Edad', 'DBT', 'IMC', 'tpo_programa', "Adherencia",
    'tpo_programa*Sexo', 'tpo_programa*Edad', 'tpo_programa*DBT', 'tpo_programa*IMC', 'tpo_programa*Adherencia'
]

mixed_both = sm.MixedLM(endog=tesis['TAS'], exog=tesis[fixed_effects], exog_re=tesis[['Intercept', 'tpo_programa']], groups=tesis['idPaciente']).fit(reml=True)
mixed_intercept = sm.MixedLM(endog=tesis['TAS'], exog=tesis[fixed_effects], exog_re=tesis[['Intercept']], groups=tesis['idPaciente']).fit(reml=True)
mixed_time = sm.MixedLM(endog=tesis['TAS'], exog=tesis[fixed_effects], exog_re=tesis[['tpo_programa']], groups=tesis['idPaciente']).fit(reml=True)

print(mixed_intercept.summary())
print((-2*mixed_intercept.llf)-(-2*mixed_both.llf))
print((-2*mixed_time.llf)-(-2*mixed_both.llf))
# Ambos efectos aleatorios son significativos

                Mixed Linear Model Regression Results
Model:                MixedLM     Dependent Variable:     TAS        
No. Observations:     3920        Method:                 REML       
No. Groups:           560         Scale:                  123.2355   
Min. group size:      7           Log-Likelihood:         -15422.3617
Max. group size:      7           Converged:              Yes        
Mean group size:      7.0                                            
---------------------------------------------------------------------
                         Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
---------------------------------------------------------------------
Intercept               117.017    4.381 26.707 0.000 108.430 125.605
Sexo                      4.006    0.922  4.344 0.000   2.198   5.814
Edad                      0.153    0.048  3.163 0.002   0.058   0.247
DBT                       2.756    1.366  2.018 0.044   0.079   5.434
IMC                       0.180    0

### Selección de modelos

In [10]:
def contrast(fixed_effects, contrast, inplace=True):
    from scipy.stats.distributions import chi2

    # Agregar variables si no están en el modelo
    for x in contrast:
        if x not in fixed_effects:
            fixed_effects.append(x)

    fixed_effects_aux = fixed_effects
    fixed_effects_reduced = [x for x in fixed_effects if x not in contrast]
    mixed_complete = sm.MixedLM(endog=tesis['TAS'], exog=tesis[fixed_effects], exog_re=tesis[['Intercept', 'tpo_programa']], groups=tesis['idPaciente']).fit(reml=False)
    mixed_reduced = sm.MixedLM(endog=tesis['TAS'], exog=tesis[fixed_effects_reduced], exog_re=tesis[['Intercept', 'tpo_programa']], groups=tesis['idPaciente']).fit(reml=False)
    print(mixed_complete.summary())
    chi_value = (-2*mixed_reduced.llf)-(-2*mixed_complete.llf)
    p_value = chi2.sf(chi_value, len(contrast))
    print('\n', contrast, 'p-value =', p_value)
    if inplace == False:
        return fixed_effects_aux
    elif p_value > 0.05 and inplace == True:
        print('\nSe remueven los efectos de', contrast)
        return fixed_effects_reduced
    else:
        print('\nEl modelo continua igual')
        return fixed_effects

In [75]:
def model(data, effects):
    mixed = sm.MixedLM(endog=data['TAS'], exog=data[effects], exog_re=data[['Intercept', 'tpo_programa']], groups=data['idPaciente']).fit(reml=False)
    return mixed

def model_gee(effects):
    mixed = sm.GEE(endog=tesis['TAS'], exog=tesis[effects], exog_re=tesis[['Intercept', 'tpo_programa']], groups=tesis['idPaciente'])
    print(mixed.summary())
    return mixed

In [85]:
def auto_select_covs(
        data: pd.DataFrame,
        effects: List[str],
        ignore_covs: Optional[List[str]],
        threshold: float = 0.05
    ) -> sm.MixedLM:
    if not ignore_covs:
        ignore_covs = []
    optimized = False
    removed_covs = []
    while not optimized:
        mixed = model(data, effects)
        p_values = mixed.pvalues.to_dict()
        for cov in ignore_covs:
            p_values.pop(cov)
        max_p_value_cov = max(p_values, key=p_values.get)
        max_p_value = p_values[max_p_value_cov]
        if max_p_value >= threshold and max_p_value_cov:
            print(f"Removing {max_p_value_cov} with p-value {max_p_value}")
            effects.remove(max_p_value_cov)
            removed_covs.append(max_p_value_cov)
            continue
        for cov in removed_covs:
            mixed = model(data, effects + [cov])
            p_values = mixed.pvalues.to_dict()
            p = p_values[cov]
            if p < threshold:
                print(f"Adding {cov} with p-value {p}")
                effects.append(cov)
                removed_covs.remove(cov)
                continue
        optimized = True
    print(mixed.summary())
    return mixed

In [86]:
fixed_effects = [
    "Intercept",
    "Sexo",
    "Edad",
    "DBT",
    "IMC",
    "tpo_programa",
    "tpo_programa*Sexo",
    "tpo_programa*Edad",
    "tpo_programa*DBT",
    "tpo_programa*IMC"
]
# fixed_effects.remove("tpo_programa*Sexo")
# fixed_effects.remove("tpo_programa*Edad")
# fixed_effects.remove("tpo_programa*IMC")
# fixed_effects.remove("IMC")

In [59]:
mod = model(fixed_effects)

                  Mixed Linear Model Regression Results
Model:                   MixedLM      Dependent Variable:      TAS        
No. Observations:        3920         Method:                  ML         
No. Groups:              560          Scale:                   115.2019   
Min. group size:         7            Log-Likelihood:          -15422.4326
Max. group size:         7            Converged:               Yes        
Mean group size:         7.0                                              
--------------------------------------------------------------------------
                              Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
--------------------------------------------------------------------------
Intercept                    115.409    3.545 32.555 0.000 108.460 122.357
Sexo                           4.008    0.755  5.311 0.000   2.529   5.488
Edad                           0.148    0.046  3.241 0.001   0.058   0.237
DBT                            2.846    1.50

In [87]:
mixed = auto_select_covs(tesis, fixed_effects, ignore_covs=["Intercept", "tpo_programa"])

Removing tpo_programa*Sexo with p-value 0.9729919157930079
Removing tpo_programa*Edad with p-value 0.4812620600193327
Removing tpo_programa*IMC with p-value 0.3157738091044451
Removing IMC with p-value 0.0713284937126934
                  Mixed Linear Model Regression Results
Model:                   MixedLM      Dependent Variable:      TAS        
No. Observations:        3920         Method:                  ML         
No. Groups:              560          Scale:                   115.2033   
Min. group size:         7            Log-Likelihood:          -15423.1708
Max. group size:         7            Converged:               Yes        
Mean group size:         7.0                                              
--------------------------------------------------------------------------
                              Coef.  Std.Err.   z    P>|z|  [0.025  0.975]
--------------------------------------------------------------------------
Intercept                    116.179    3.564 32

In [13]:
mixed = sm.MixedLM(endog=tesis['TAS'], exog=tesis[["Intercept", "tpo_programa", "Adherencia", "tpo_programa*Adherencia"]], exog_re=tesis[['Intercept', 'tpo_programa']], groups=tesis['idPaciente']).fit(reml=False)
print(mixed.summary())

                   Mixed Linear Model Regression Results
Model:                  MixedLM       Dependent Variable:       TAS        
No. Observations:       3920          Method:                   ML         
No. Groups:             560           Scale:                    113.2404   
Min. group size:        7             Log-Likelihood:           -15413.3124
Max. group size:        7             Converged:                Yes        
Mean group size:        7.0                                                
---------------------------------------------------------------------------
                              Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
---------------------------------------------------------------------------
Intercept                    134.286    0.902 148.849 0.000 132.518 136.054
tpo_programa                   0.385    0.240   1.605 0.109  -0.085   0.856
Adherencia                    -2.243    0.924  -2.427 0.015  -4.054  -0.432
tpo_programa*Adherencia       -

In [14]:
mixed = sm.MixedLM.from_formula(data=tesis, formula="TAS ~ tpo_programa + tpo_programa*Adherencia", groups="idPaciente", re_formula="~tpo_programa").fit(reml=False)
print(mixed.summary())

                   Mixed Linear Model Regression Results
Model:                   MixedLM       Dependent Variable:       TAS        
No. Observations:        3920          Method:                   ML         
No. Groups:              560           Scale:                    113.2404   
Min. group size:         7             Log-Likelihood:           -15413.3124
Max. group size:         7             Converged:                Yes        
Mean group size:         7.0                                                
----------------------------------------------------------------------------
                               Coef.  Std.Err.    z    P>|z|  [0.025  0.975]
----------------------------------------------------------------------------
Intercept                     134.286    0.902 148.849 0.000 132.518 136.054
tpo_programa                    0.385    0.240   1.605 0.109  -0.085   0.856
Adherencia                     -2.243    0.924  -2.427 0.015  -4.054  -0.432
tpo_programa:Adhere

In [15]:
mixed.cov_re

,idPaciente,tpo_programa
idPaciente,99.030960,-8.288801
tpo_programa,-8.288801,2.146411
